# Installing required dependencies and importing libraries

In [ ]:
!pip install opencv-python==3.4.2.17
!pip install opencv-contrib-python==3.4.2.17
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os
from natsort import natsorted

     |████████████████████████████████| 25.0 MB 1.2 MB/s 
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 30.6 MB 1.2 MB/s 
  Attempting uninstall: opencv-contrib-python
    Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


## Mounting Google drive for storing data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Image Matching using SIFT, SURF and ORB

Image matching is an important concept in computer vision and object recognition. The images of the same subject can be acquired from different angles, lighting and scales. Therefore, it is best to find features that are descriptive and invariant to the aforementioned variations in order to categorize the images correctly.

SIFT helps locate the local features in an image, commonly known as the ‘keypoints‘ of the image. These keypoints are scale & rotation invariant that can be used for various computer vision applications, like image matching, object detection, scene detection, etc.

We can also use the keypoints generated using feature extractors like SIFT, SURF, ORB and arcFace as features for the image during model training. The major advantage of SIFT features, over edge features or hog features, is that they are not affected by the size or orientation of the image.

In the following cell we have compared the images using SIFT, SURF and ORB models and then stored the results in a text file. Here I have done nxn mapping, irrespective of the sessions which I will filter it later on while reading the file into local variables for error analysis.

In [ ]:
file_sift = open("gdrive/MyDrive/DeepLearningAssignment/Result_sift.txt", "w")
file_surf = open("gdrive/MyDrive/DeepLearningAssignment/Result_surf.txt", "w")
file_orb = open("gdrive/MyDrive/DeepLearningAssignment/Result_orb.txt", "w")
path1 = 'gdrive/MyDrive/face_data'

sift = cv2.xfeatures2d.SIFT_create()
surf = cv2.xfeatures2d.SURF_create()
orb = cv2.ORB_create(15000)

bf = cv2.BFMatcher()
clahe = cv2.createCLAHE(clipLimit = 2.0, tileGridSize = (8,8))
for images in natsorted(os.listdir(path1)):
    str1 = str(images)
    img1 = os.path.join(path1, images)
    # print(img1)
    path_img1 = img1
    img1 = cv2.imread(img1, cv2.IMREAD_GRAYSCALE)
    img1 = clahe.apply(img1)
    kp1_sift, des1_sift = sift.detectAndCompute(img1,None)
    kp1_surf, des1_surf = surf.detectAndCompute(img1,None)
    kp1_orb, des1_orb = orb.detectAndCompute(img1,None)
    if len(kp1_sift)>0 and len(kp1_surf)>0 and len(kp1_orb)>0:
        for image in natsorted(os.listdir(path1)):
            img2 = os.path.join(path1, image)
            #    print(img2)
            str2 = str(image)
            if (img1 != img2): # queryImage
                img2 = cv2.imread(img2, cv2.IMREAD_GRAYSCALE) # trainImage
                img2 = clahe.apply(img2)
                kp2_sift, des2_sift = sift.detectAndCompute(img2,None)
                kp2_surf, des2_surf = surf.detectAndCompute(img2,None)
                kp2_orb, des2_orb = orb.detectAndCompute(img2,None)
                # Initiate SIFT detector
                # sift = cv2.SIFT()            
                # find the keypoints and descriptors with SIFT
                # BFMatcher with default params
                matches_sift = bf.knnMatch(des1_sift,des2_sift, k=2)
                matches_surf = bf.knnMatch(des1_surf, des2_surf, k=2)
                if type(des1_orb) != type(None) and type(des2_orb) != type(None) and des1_orb.shape[1] == des2_orb.shape[1]:
                    matches_orb = bf.knnMatch(des1_orb, des2_orb, k=2)
                dis_sift = bf.match(des1_sift,des2_sift)
                dis_surf = bf.match(des1_surf,des2_surf)
                if type(des1_orb) != type(None) and type(des2_orb) != type(None) and des1_orb.shape[1] == des2_orb.shape[1]:
                    dis_orb = bf.match(des1_orb,des2_orb)
            #----------------------------------------------
                number_keypoints_sift = 0
                if len(kp1_sift) <= len(kp2_sift):
                    number_keypoints_sift = len(kp1_sift)
                else:
                    number_keypoints_sift = len(kp2_sift)
                good_sift = []
                for m,n in matches_sift:
                    if m.distance < 0.7*n.distance:
                        good_sift.append([m])

                number_keypoints_surf = 0
                if len(kp1_surf) <= len(kp2_surf):
                    number_keypoints_surf = len(kp1_surf)
                else:
                    number_keypoints_surf = len(kp2_surf)
                good_surf = []
                for m,n in matches_surf:
                   if m.distance < 0.7*n.distance:
                      good_surf.append([m])

                number_keypoints_orb = 0
                if len(kp1_sift) <= len(kp2_orb):
                    number_keypoints_orb = len(kp1_orb)
                else:
                    number_keypoints_orb = len(kp2_orb)
                good_orb = []
                for i, pair in enumerate(matches_orb):
                    try:
                        m, n = pair
                        if m.distance < 0.7*n.distance:
                            good_orb.append(m)
                    except ValueError:
                        continue
            #print("Keypoints 1ST Image: " + str(len(kp1)))
            #print("Keypoints 2ND Image: " + str(len(kp2)))
            #print("GOOD Matches:", len(good))
                score_sift = len(good_sift) / number_keypoints_sift
                file_sift.write(str1 + " " + str2 + " " + str(score_sift)+"\n")
                score_surf = len(good_surf) / number_keypoints_surf
                file_surf.write(str1 + " " + str2 + " " + str(score_surf)+"\n")
                if number_keypoints_orb != 0:
                    score_orb = len(good_orb) / number_keypoints_orb
                    file_orb.write(str1 + " " + str2 + " " + str(score_orb)+"\n")
                    
            
                
                
            
file_sift.close()
file_surf.close()
file_orb.close()
print("all well")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


all well
